In [1]:
print("starting imports")
# Philipp Bartz 2025

print("1")

import os
import velox
import time
import custom_pathlibrary as cpath
import custom_filehandler as cfile
import custom_measurement as cmeasure
import custom_waferprober as cwafer
import custom_logging as clog
import custom_analyzer as canal
import logging
import serial 
import typing
import json



def is_running(*, logger):
    ''' Check if the program is running based on the contents of activate.txt.
        If the file contains "True", the program continues.
        If it contains "Pause", the program enters a loop checking every 60 seconds.
        If it contains anything else or the file is not found, the program stops.
        Returns:

            True if the program should continue running,
            
            False if it should stop.'''
    try:
        with open("activate.txt", 'r') as f:
            contents = f.read()
        if "True" in contents:
            return True
        elif "Pause" in contents:
            logger.warning("activate.txt contains 'Pause'. Stopping the program temporarily.")
            print("is_running: Pause")


            while True:         # the Pause mode checks every 60 seconds if the activate.txt file has changed
                time.sleep(60)
                try:
                    with open("activate.txt", 'r') as f:
                        contents = f.read()
                    if "True" in contents:
                        return True
                    elif "Pause" in contents:
                        logger.warning("activate.txt contains 'Pause'. Stopping the program temporarily.")
                        print("is_running: Pause")
                    else:
                        logger.critical("activate.txt does not contain 'True'. Stopping the program.")
                        print("is_running: False")
                        return False
                except FileNotFoundError:
                    logger.critical("File not found: activate.txt")
                    print("is_running: File not found")
                    return False
                except Exception as e:
                    logger.critical(f"Error reading activate.txt: {e}")
                    print("is_running: Error reading")
                    return False

        else:
            logger.critical("activate.txt does not contain 'True'. Stopping the program.")
            print("is_running: False")
            return False
    except FileNotFoundError:
        logger.critical("File not found: activate.txt")
        print("is_running: File not found")
        return False
    except Exception as e:
        logger.critical(f"Error reading activate.txt: {e}")
        print("is_running: Error reading")
        return False



# Main control loop
def main(*, 
         only_init=False, 
         only_single_subdie=False, 
         path_to_config_file: str = "",

        ###    the following parameters are the fallback/default values if the config file does not contain them ###
         chuck_target_temperature=25, # target temperature for the chuck in degrees Celsius, set to 25 for room temperature
         force_temperature=True, 
         ):

    '''
    load settings from file 

    '''
    if not os.path.isfile(path_to_config_file):
        print(f"Error: File {path_to_config_file} does not exist.")
    
    with open(path_to_config_file, 'r', encoding='utf-8') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file {path_to_config_file}: {e}")
            return None
        except Exception as e:
            print(f"An unexpected error occurred while loading JSON file {path_to_config_file}: {e}")
            return None
    chuck_target_temperature = data.get("chuck_target_temperature", 25)
    device_port = data.get("device_port", "/dev/ttyUSB0")
    extra = data.get("extra", "")
    measurements_per_diode = data.get("measurements_per_diode", 1)
    movement_time = data.get("movement_time", 0)
    plotting = data.get("plotting", False)
    tag_1 = data.get("tag_1", "")
    tag_2 = data.get("tag_2", "")
    tag_3 = data.get("tag_3", "")
    force_temperature = data.get("force_temperature", True)
    only_single_subdie = data.get("only_single_subdie", False)
    waferprober_ip = data.get("waferprober_ip", "192.168.255.1") # IP address of the wafer prober
    start_first_die = data.get("start_first_die", True)

    
    
    ###     Initialization     ###
    wafer_folder = cfile.create_folder()
    timestr = time.strftime("%Y_%m_%d-%H_%M_%S")
    tmptime = time.time()
    tmptime_alive_signal = time.time()
    tmptime_failed_measurements = time.time()
    failed_measurement_timestamps = []

    logger = clog.CustomLogger.setup_logger(name="custom_logger", log_file=f"{wafer_folder}/event.log")
    logger.critical(f"Start time: {timestr}")

    settings = cfile.save_to_json(folder="")
    settings.add(name="time_at_start", value=timestr)
    settings.add(name="device_port", value=device_port)
    settings.add(name="waferprober_ip", value=waferprober_ip)
    settings.add(name="plotting", value=plotting)
    settings.add(name="movement_time", value=movement_time)
    settings.add(name="chuck_target_temperature", value=chuck_target_temperature)
    settings.add(name="extra", value=extra)
    settings.add(name="wafer_folder", value=wafer_folder)
    settings.add(name="tag_1", value=tag_1)
    settings.add(name="tag_2", value=tag_2)
    settings.add(name="tag_3", value=tag_3)
    settings.add(name="measurements_per_diode", value=measurements_per_diode)
    settings.save(folder=wafer_folder, logger=logger)
    if not is_running(logger=logger):
        return

    # initialize Keithley
    cmeasure.init_IV(dev=device_port, logger=logger)

    # Connect to Velox Message Server
    print("connecting MSG Server....")
    msgServer = cwafer.connect_to_message_server(ip=waferprober_ip, logger=logger)
    print("connected")
    if only_init:
        return msgServer, logger
    if not msgServer:
        logger.error("Could not connect to Velox Message Server.")
        return
    
    
    
    cwafer.register_applications(logger=logger)

    
    cwafer.set_heater_temp(msgServer=msgServer, temperature=chuck_target_temperature, logger=logger)
    if force_temperature:
        current_temperature = cwafer.get_chuck_temperature(msgServer=msgServer, logger=logger)
        while current_temperature != chuck_target_temperature:
            time.sleep(5) # wait for 5 seconds before checking again if the chuck has reached the target temperature
            current_temperature = cwafer.get_chuck_temperature(msgServer=msgServer, logger=logger)

    if wafer_folder is None:
        logger.error("Wafer folder is not created. Exiting..")
        return
    cwafer.set_scope_light(msgServer=msgServer, light_on=False, logger=logger) # turn off Scope Light
    cwafer.set_quiet_mode_scope(msgServer=msgServer, quiet_mode=True, logger=logger) # set quiet mode for scope
    if start_first_die:
        answer = msgServer.sendSciCommand("StepFirstDie")
        logger.info(f"Stepped to first Die. Answer: {answer}")
    
    
    ###     End of initialization     ###


    
    while cwafer.step_to_next_subdie(msgServer=msgServer, logger=logger) and is_running(logger=logger):
        cwafer.set_quiet_mode_scope(msgServer=msgServer, quiet_mode=True, logger=logger)   
        logger.info(f"Starting new subdie movement and measurement")
        path = cpath.PLib()
        pathsteps = path.find_path(msgServer=msgServer,extra=extra, logger=logger)
        for i in pathsteps:
            probe = msgServer.sendSciCommand("MoveProbeSeparation", 1)
            logger.path(f"{probe} on seperation height") # type: ignore
            time.sleep(movement_time)
            logger.path(f"Moving Probe to {i[0]}, {i[1]}") # type: ignore
            msgServer.sendSciCommand("MoveProbe", 1, i[0], i[1])
            time.sleep(movement_time)
            if not is_running(logger=logger):
                logger.critical("Stopping Path due to is_running check.")
                break
            
            if i[2] == "PAD":   # if the pathpoint is a tagged PAD, it is a measurement point
                diode_Nr = i[3]
                logger.lowering(f"Lowering Probe to contact Hight") # type: ignore
                msgServer.sendSciCommand("MoveProbeContact", 1)

                cwafer.set_quiet_mode_motor(msgServer=msgServer, quiet_mode=True, logger=logger)
                logger.measurement(f"STARTING MEASUREMENT of diode {diode_Nr}") # type: ignore

            ###          MEASUREMENT        ###
                for i in range(measurements_per_diode):
                    measurement_no = i + 1
                    logger.measurement(f"Measurement {measurement_no} of {measurements_per_diode}") # type: ignore
                    cmeasure.measure(msgServer=msgServer, 
                                     device_port=device_port, 
                                     folder=wafer_folder, 
                                     diode_Nr=diode_Nr, 
                                     settings=settings, 
                                     logger=logger, 
                                     plotting=plotting, 
                                     extra=extra, 
                                     measurements_per_diode=measurements_per_diode, 
                                     measurement_no=measurement_no, 
                                     path_to_config_file=path_to_config_file,
                                    failed_measurement_timestamps=failed_measurement_timestamps)
                    logger.measurement(f"MEASUREMENT DONE") # type: ignore
                    if (tmptime_alive_signal + 14400) < time.time():
                        tmptime_alive_signal = time.time()
                        print("still running")
                        logger.critical(f"Waferprober still running. Measurement no {i} of {measurements_per_diode} at diode_Nr {diode_Nr}")
                    if (tmptime_failed_measurements + 3600) < time.time():
                        tmptime_failed_measurements = time.time()
                        if len(failed_measurement_timestamps) > 7:
                            logger.critical(f"IMPORTANT!- The Waferprober measured at least 7 measurements with only noise! This might be a problem! (- clearing list)")
                            failed_measurement_timestamps = []
                        else:
                            failed_measurement_timestamps = []
                    if not is_running(logger=logger):
                        logger.critical("Stopping Measurement due to is_running check.")
                        break
            ###          END OF MEASUREMENT        ###

                cwafer.set_quiet_mode_motor(msgServer=msgServer, quiet_mode=False, logger=logger)
        logger.path(f"ALL MOVEMENTS DONE for this Subdie")
        if only_single_subdie:
            logger.info("Only single subdie measurement requested, stopping after this subdie.")
            break
        if not is_running(logger=logger):
            logger.critical("Stopping Measurement Loop due to is_running check.")
            break


###     END OF PROBE AND MEASURE     ###)
    logger.info(f"Probe and measure completed successfully.")
    timestr = time.strftime("%Y_%m_%d-%H_%M_%S")
    settings.add(name="time_at_end", value=timestr)
    settings.add(name="time_elapsed", value=time.time() - tmptime)
    settings.add(name="finished_successfully", value=True)
    settings.save(folder=wafer_folder, logger=logger)

    cwafer.set_scope_light(msgServer=msgServer, light_on=True, logger=logger)
    cwafer.set_quiet_mode_scope(msgServer=msgServer, quiet_mode=False, logger=logger)


    answer = msgServer.sendSciCommand("StepFirstDie")
    logger.critical(f"END OF main(), wafertest finished, stepped to first Die. Answer: {answer}")
    



print("init finished")

starting imports
1
init finished


## Task Scheduler


In [ ]:
## Task Scheduler   ##



global_logger = clog.CustomLogger.setup_logger(name="global_logger", log_file="global_logs/global_event.log")

    ## move Settings file
if not os.path.exists(f"task_list/finished"):
    os.makedirs(f"task_list/finished")
while is_running(logger=global_logger):
    task_list = os.listdir("task_list")
    task_list.sort()
    for i in task_list:
        if not i.endswith(".json"):
            task_list.remove(i)
    if len(task_list) == 0:
        global_logger.info("No tasks found in task_list folder, waiting for new tasks. (20s)")
        time.sleep(20)
    else:
        global_logger.info(f"Found {len(task_list)} tasks in task_list folder, starting first task.")
        global_logger.info  (f"Starting task {task_list[0]}")
        path_to_config_file = f"task_list/{task_list[0]}"



        # main() --- IGNORE ---
        time.sleep(3)
        main(path_to_config_file=path_to_config_file)


        
        # move config file to finished folder
        if not os.path.exists("task_list/finished"):
            os.makedirs("task_list/finished")
        if is_running(logger=global_logger):
            if os.path.isfile(f"task_list/finished/{task_list[0]}"):
                global_logger.warning(f"File {task_list[0]} already exists in finished folder, timestamping current file.")
                timestr = time.strftime("%Y_%m_%d-%H_%M_%S")
                path = f"task_list/finished/{timestr}_{task_list[0]}"
            else:
                path = f"task_list/finished/{task_list[0]}"
            global_logger.info(f"Moving {path_to_config_file} to {path}")
            os.rename(path_to_config_file, path)

print(time.strftime("%Y_%m_%d-%H_%M_%S"))

Created folder: data/Wafer_at_2025_07_24-15_46_45
💥 CRITICAL - Start time: 2025_07_24-15_46_45
💥 CRITICAL - Saving settings to data/Wafer_at_2025_07_24-15_46_45/settings_dump.json
💥 CRITICAL - Settings: {'time_at_start': '2025_07_24-15_46_45', 'device_port': '/dev/ttyUSB0', 'waferprober_ip': '192.168.255.1', 'plotting': False, 'movement_time': 0, 'chuck_target_temperature': 60, 'extra': '_short', 'wafer_folder': 'data/Wafer_at_2025_07_24-15_46_45', 'tag_1': 'temperature_test', 'tag_2': '60C', 'tag_3': '', 'measurements_per_diode': 1}
connecting MSG Server....
connected
⬇️ LOWERING - Lowering Probe to contact Hight
readline, decode
readline, decode
⬇️ LOWERING - Lowering Probe to contact Hight
readline, decode
readline, decode
⬇️ LOWERING - Lowering Probe to contact Hight
readline, decode
readline, decode
⬇️ LOWERING - Lowering Probe to contact Hight
readline, decode
readline, decode
⬇️ LOWERING - Lowering Probe to contact Hight
readline, decode
readline, decode
⬇️ LOWERING - Lowering P

In [ ]:
## test single subdie at static temperature of 25C 
main( only_single_subdie = True)


# test section
->

In [ ]:
answer = msgServer.sendSciCommand("StepFirstDie")
print(answer)

In [ ]:
movement_time = 2
path = cpath.PLib()
pathsteps = path.find_path(msgServer=msgServer,extra="", logger=logger)
for i in pathsteps:
    probe = msgServer.sendSciCommand("MoveProbeSeparation", 1)
    msgServer.sendSciCommand("MoveProbe", 1, i[0], i[1])
    if i[2] == "PAD":   # if the pathpoint is a tagged PAD, it is a measurement point
                diode_Nr = i[3]
                msgServer.sendSciCommand("MoveProbeContact", 1)
                print(f"i[0]: {i[0]}, i[1]: {i[1]}")
                time.sleep(2)
    